In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython


In [12]:
%config Completer.use_jedi = False

In [2]:
import sys
sys.path.append('../')

In [3]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

### Canary

In [4]:
%%time
canary_p = '../../../data/CANARY/CANARY202104'
pmpm_p = '../../../temp/PMPM.p'
preg_p = '../../../data/pregnancies'

preg = pq.read_table(preg_p).to_pandas()
canary = pq.read_table(canary_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)




CPU times: user 3.02 s, sys: 5.36 s, total: 8.38 s
Wall time: 8.28 s


### CCM

In [5]:
%%time
pmpm_ccm_path = '../../../temp/pmpm_ccm.csv'
ref_ccm_path = '../../../temp/ref.csv'


pmpm_ccm = pd.read_csv(pmpm_ccm_path)
ref_ccm = pd.read_csv(ref_ccm_path)


CPU times: user 1min 33s, sys: 13.2 s, total: 1min 46s
Wall time: 1min 46s


In [6]:
!rm -r ../../../temp/pmpm_post18_cmm.csv
!rm -r ../../../temp/pmpm_ccm.csv
!rm -r ../../../temp/ref.csv
!rm  -r ../../../temp/PMPM.p

In [7]:
pmpm_ccm.columns = [col.upper() for col in pmpm_ccm.columns]
pmpm_ccm.rename(columns = {'PERSON_ID':'PERS_ID'}, inplace = True)
cols = [col for col in pmpm.columns if  col != 'MEDICAID_LOB']
pmpm_ccm = pmpm_ccm[cols]

In [8]:
pmpm_ccm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  \
0  2612620  201510        M            75.70       0.00       0.0       0.0        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0   
1  2613447  201510        M            75.54    2159.51       0.0       0.0        0             0                  0       0          0                    0             0          0              1          0           0       1                 0            1                  0                  0                     0        0                   0              0                        0                  0            1                0          7                      0            0                0            0                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                0               0             0            1               0             0         0               0            0                   0                      0                      0           0                0   
2  2613818  201510        M            75.46     176.44       0.0       0.0        0             0                  0       0          1                    0             0          0              0          0           0       0                 0            0                  0                  1                     0        0                   0              1                        0                  0            1                0          7                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           1                0   

In [9]:
pmpm_ccm.MYR = pmpm_ccm.MYR.astype(int)
pmpm_ccm.PERS_ID = pmpm_ccm.PERS_ID.astype(str)
pmpm_ccm.AGE_AT_MIDMONTH = pmpm_ccm.AGE_AT_MIDMONTH.astype(int)
pmpm_ccm.ALLOW_AMT = pmpm_ccm.ALLOW_AMT.astype(int)

In [10]:
pmpm_ccm = pmpm_ccm.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)

In [11]:
ref_ccm = ref_ccm[~ref_ccm.ref_create_date.isin(['Member (Self Referral)'])].reset_index(drop = True)
ref_ccm['REGISTER_DATE'] = pd.to_datetime(ref_ccm['ref_create_date'], format = '%Y-%m-%d')
ref_ccm.rename(columns = {'person_id':'PERS_ID'}, inplace = True)
ref_ccm.sort_values(by = ['PERS_ID', 'REGISTER_DATE'], inplace = True)
ref_ccm.dropna(axis = 0, subset = ['PERS_ID', 'REGISTER_DATE'], inplace = True)
ref_ccm = ref_ccm.groupby(by = 'PERS_ID').apply(lambda x: x.iloc[0]).reset_index(drop = True)

In [14]:
ref_ccm['REGISTER_DATE'] 

0       2014-02-12
1       2015-02-17
2       2014-10-22
3       2014-12-16
4       2019-11-21
           ...    
42045   2020-06-23
42046   2020-07-13
42047   2020-07-14
42048   2020-07-07
42049   2020-07-15
Name: REGISTER_DATE, Length: 42050, dtype: datetime64[ns]

In [15]:
#if you want to change register date
offset = 0
ref_ccm['REGISTER_DATE']  = ref_ccm['REGISTER_DATE'] + pd.DateOffset(months = offset)

In [17]:
pmpm_ccm.to_pickle('../../../temp/pmpm_ccm.p')
ref_ccm.to_pickle('../../../temp/ref_ccm.p')